In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from typing import List ,Dict, Tuple
import numpy as np

In [6]:
train = pd.read_csv('./JobCare_data/train.csv')

In [7]:
nominal_cols = ['person_attribute_a_1','person_attribute_b','person_prefer_e','contents_attribute_e']

In [ ]:
# 오버피팅이 자주 발생하는 mean encoding 이므로 cross validation 과 정규화 같이 사용한다=
# 변환 하고자 하는 범주형 변수 선택
# 범주형 변수 그룹화 -> 타깃 변수 총합 합계
# 범주형 변수 그룹화 타깃 빈도수 합계
# 총합을 카운트로 나누고 본래 범주 값에 업데이트
# 여러가지 방법으로 적용 가능하다
# 비슷한 범주 사이에 있는 관계 표현 특징, 범주와 타깃사이에만 국한된다
# 범주가 많은 경우 이 방법은 데이터를 훨씬 더 단순화 한다

In [8]:
for col in nominal_cols:
    if 'attribute' in col:
        name = col[col.index('attribute') + len('attribute')+1:]
        var_name = 'attr_{}_mean_encode'.format(name)
        locals()[var_name] = train.groupby(col)["target"].mean()
    elif 'prefer' in col:
        name = col[col.index('prefer') + len('prefer')+1:]
        var_name = 'prefer_{}_mean_encode'.format(name)
        locals()[var_name] = train.groupby(col)["target"].mean()
    train.loc[:,var_name] = train[col].map(locals()[var_name])

In [9]:
target = train.target
train = train.drop('target',axis=1)

In [10]:
train = train.drop(['contents_open_dt','id'],axis=1)

In [11]:
x_train, x_valid, y_train, y_valid = train_test_split(train,target , test_size=0.3 , shuffle=True, stratify=target, random_state=34)
rf= RandomForestClassifier(n_estimators=150 , max_depth=8 , min_samples_split=4, max_features=0.2, n_jobs=-1 ,random_state=0)
rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=8, max_features=0.2, min_samples_split=4,
                       n_estimators=150, n_jobs=-1, random_state=0)

In [57]:

# 과적합일진 모르지만 그래도많이올랐따 만족할 수 없음
# 0.57정도에서 0.59 정도로 mean encoding 으로 상승

(y_valid == rf.predict(x_valid)).mean()

0.5970076899578978

In [ ]:
#catboost + cv 5fold + threshold

In [37]:
# 전역 변수
SEED = 42
DATA_PATH = './JobCare_data/'

In [29]:
train_data = pd.read_csv(f'{DATA_PATH}train.csv')
test_data = pd.read_csv(f'{DATA_PATH}test.csv')

In [ ]:
code_d = pd.read_csv(f'{DATA_PATH}속성_D_코드.csv').iloc[:,:-1]
code_h = pd.read_csv(f'{DATA_PATH}속성_H_코드.csv')
code_l = pd.read_csv(f'{DATA_PATH}속성_L_코드.csv')

In [19]:
code_d.columns  = ["attribute_d", "attribute_d_d" , "attribute_d_s", "attribute_d_m" , "attribute_d_l"]
code_h.columns = ["attribute_h" , "attribute_h_p"]
code_l.columns = ["attribute_l", "attribute_l_d", "attribute_l_s" , "attribute_l_m", "attribute_l_l"]


In [ ]:
col

In [49]:
code_d.add_prefix("contents_attribute_e_")

,contents_attribute_e_attribute_d,contents_attribute_e_attribute_d_d,contents_attribute_e_attribute_d_s,contents_attribute_e_attribute_d_m,contents_attribute_e_attribute_d_l
0,4,4,2,3,1
1,5,5,2,3,1
2,7,7,2,6,1
3,8,8,2,6,1
4,9,8,2,6,1
...,...,...,...,...,...
1109,1254,1254,1235,1254,1235
1110,1255,1254,1235,1254,1235
1111,1256,1254,1235,1254,1235
1112,1257,1254,1235,1254,1235


,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,...,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,target
0,0,True,True,True,False,False,False,1,4,3,...,2,1608,275,1,4,139,618822,354805,2020-01-17 12:09:36,1
1,1,False,False,False,True,True,False,1,3,4,...,2,1608,275,1,4,133,571659,346213,2020-06-18 17:48:52,0
2,2,False,False,False,True,False,False,2,0,3,...,1,1600,94,1,4,53,399816,206408,2020-07-08 20:00:10,0
3,3,False,False,False,True,False,False,2,0,2,...,2,1608,275,5,3,74,827967,572323,2020-01-13 18:09:34,0
4,4,True,True,True,False,False,False,1,3,4,...,2,1608,275,1,4,74,831614,573899,2020-03-09 20:39:22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,501946,False,False,False,True,False,False,1,1,2,...,2,354,147,1,5,65,503156,285850,2020-03-13 12:55:52,1
501947,501947,True,True,False,True,False,False,1,6,2,...,2,163,120,1,4,142,676255,456996,2020-01-20 11:51:51,1
501948,501948,True,True,True,True,False,False,1,7,4,...,2,438,147,2,7,65,484528,293258,2020-08-05 17:27:24,1
501949,501949,True,False,False,True,False,False,1,1,2,...,2,660,147,3,4,259,456330,273797,2020-06-15 09:23:21,1


In [ ]:
train_data

In [54]:
# 하나는 있는 거니까 , left merge는 있는 컬럼키면 그대로두고
# 없는 컬럼 키면 그대로 붙이니까 merge가 가능하다

In [56]:
def merge_codes(df:pd.DataFrame, df_code:pd.DataFrame, col:str) -> pd.DataFrame:
    df = df.copy()
    df_code = df_code.copy()
    df_code = df_code.add_prefix(f"{col}_")
    df_code.columns.values[0] = col
    print('df :' ,df)
    print('df_code : ', df_code)
    print('merge data : ',pd.merge(df,df_code , how='left', on= col) )
    return pd.merge(df,df_code , how='left', on= col)

typing.List[typing.Tuple[str, pandas.core.frame.DataFrame]]

In [ ]:
List[Tuple[str,pd.DataFrame]]

In [52]:
# type hinting , type annotation 타입에 대한 메타 정보를 제공한다 중대규모 이상프로젝트에서는 컴파일 위험 에러
def preprocess_data(
                    df:pd.DataFrame,is_train:bool = True,
                    cols_merge:List[Tuple[str,pd.DataFrame]] = []  ,
                    cols_equi:List[Tuple[str,str]]= [] ,
                    cols_drop:List[str] = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt"]
                    )->Tuple[pd.DataFrame,np.ndarray]:

    y_data = None

    if is_train:
        y_data = df["target"].to_numpy()
        df = df.drop(columns="target")

    for col , df_code in cols_merge:
        df = merge_codes(df, df_code, col)

    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] = df[cols].astype(int)

    for col1, col2 in cols_equi:
        df[f"{col1}_{col2}"] = (df[col1] == df[col2]).astype(int)
    df = df.drop(columns=cols_drop)

    return (df,y_data)

In [ ]:
cols_merge = [
              ("person_prefer_d_1" , code_d),
              ("person_prefer_d_2" , code_d),
              ("person_prefer_d_3" , code_d),
              ("contents_attribute_d" , code_d),
              ("person_prefer_h_1" , code_h),
              ("person_prefer_h_2" , code_h),
              ("person_prefer_h_3" , code_h),
              ("contents_attribute_h" , code_h),
              ("contents_attribute_l" , code_l),
]

In [27]:

# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [
    ("contents_attribute_c","person_prefer_c"),
    ("contents_attribute_e","person_prefer_e"),
    ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_l" , "contents_attribute_d_attribute_d_l"),
    ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_l" , "contents_attribute_d_attribute_d_l"),
    ("person_prefer_h_1_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_2_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_3_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
]

# 학습에 필요없는 컬럼 리스트
cols_drop = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt", "contents_rn", ]

In [55]:
x_train, y_train = preprocess_data(train_data, cols_merge = cols_merge , cols_equi= cols_equi , cols_drop = cols_drop)
x_test, _ = preprocess_data(test_data,is_train = False, cols_merge = cols_merge , cols_equi= cols_equi  , cols_drop = cols_drop)
x_train.shape , y_train.shape , x_test.shape

df :             id  d_l_match_yn  d_m_match_yn  d_s_match_yn  h_l_match_yn  \
0            0          True          True          True         False   
1            1         False         False         False          True   
2            2         False         False         False          True   
3            3         False         False         False          True   
4            4          True          True          True         False   
...        ...           ...           ...           ...           ...   
501946  501946         False         False         False          True   
501947  501947          True          True         False          True   
501948  501948          True          True          True          True   
501949  501949          True         False         False          True   
501950  501950          True          True          True          True   

        h_m_match_yn  h_s_match_yn  person_attribute_a  person_attribute_a_1  \
0              False      

KeyboardInterrupt: 

In [ ]:
# 범주형 컬럼 리스트

In [33]:
cat_features = x_train.columns[x_train.nunique() > 2].tolist()

In [62]:
x_train

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,...,contents_attribute_e_person_prefer_e,person_prefer_d_2_attribute_d_s_contents_attribute_d_attribute_d_s,person_prefer_d_2_attribute_d_m_contents_attribute_d_attribute_d_m,person_prefer_d_2_attribute_d_l_contents_attribute_d_attribute_d_l,person_prefer_d_3_attribute_d_s_contents_attribute_d_attribute_d_s,person_prefer_d_3_attribute_d_m_contents_attribute_d_attribute_d_m,person_prefer_d_3_attribute_d_l_contents_attribute_d_attribute_d_l,person_prefer_h_1_attribute_h_p_contents_attribute_h_attribute_h_p,person_prefer_h_2_attribute_h_p_contents_attribute_h_attribute_h_p,person_prefer_h_3_attribute_h_p_contents_attribute_h_attribute_h_p
0,1,1,1,0,0,0,1,4,3,5,...,0,0,0,1,0,0,1,0,1,0
1,0,0,0,1,1,0,1,3,4,1,...,1,0,0,0,0,0,0,1,1,1
2,0,0,0,1,0,0,2,0,3,5,...,0,1,0,1,0,0,0,1,0,1
3,0,0,0,1,0,0,2,0,2,5,...,1,0,0,0,0,0,0,1,0,0
4,1,1,1,0,0,0,1,3,4,5,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,0,0,0,1,0,0,1,1,2,2,...,0,0,0,0,0,0,0,1,0,0
501947,1,1,0,1,0,0,1,6,2,1,...,1,1,1,1,1,1,1,1,1,1
501948,1,1,1,1,0,0,1,7,4,1,...,0,0,0,1,1,1,1,1,0,0
501949,1,0,0,1,0,0,1,1,2,1,...,1,1,1,1,1,1,1,1,0,1


In [34]:
cat_features

['person_attribute_a_1',
 'person_attribute_b',
 'person_prefer_c',
 'person_prefer_d_1',
 'person_prefer_d_2',
 'person_prefer_d_3',
 'person_prefer_e',
 'person_prefer_h_1',
 'person_prefer_h_2',
 'person_prefer_h_3',
 'contents_attribute_i',
 'contents_attribute_a',
 'contents_attribute_j_1',
 'contents_attribute_c',
 'contents_attribute_l',
 'contents_attribute_d',
 'contents_attribute_m',
 'contents_attribute_e',
 'contents_attribute_h',
 'person_rn',
 'person_prefer_d_1_attribute_d_d',
 'person_prefer_d_1_attribute_d_s',
 'person_prefer_d_1_attribute_d_m',
 'person_prefer_d_1_attribute_d_l',
 'person_prefer_d_2_attribute_d_d',
 'person_prefer_d_2_attribute_d_s',
 'person_prefer_d_2_attribute_d_m',
 'person_prefer_d_2_attribute_d_l',
 'person_prefer_d_3_attribute_d_d',
 'person_prefer_d_3_attribute_d_s',
 'person_prefer_d_3_attribute_d_m',
 'person_prefer_d_3_attribute_d_l',
 'contents_attribute_d_attribute_d_d',
 'contents_attribute_d_attribute_d_s',
 'contents_attribute_d_attrib

In [ ]:

from sklearn.model_selection import StratifiedKFold , KFold

In [38]:
is_holdout = False
n_splits = 5
iterations = 3000
patiencex = 50

cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

In [41]:
from catboost import Pool,CatBoostClassifier

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.086395
0:	learn: 0.5918078	test: 0.5960459	best: 0.5960459 (0)	total: 2.27s	remaining: 1h 53m 20s
100:	learn: 0.6633124	test: 0.6885151	best: 0.6886768 (99)	total: 2m 54s	remaining: 1h 23m 20s
200:	learn: 0.6694600	test: 0.6917862	best: 0.6919422 (160)	total: 5m 43s	remaining: 1h 19m 38s
300:	learn: 0.6733411	test: 0.6930620	best: 0.6932168 (288)	total: 8m 53s	remaining: 1h 19m 44s
400:	learn: 0.6758620	test: 0.6938391	best: 0.6941906 (393)	total: 11m 24s	remaining: 1h 13m 56s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6941906097
bestIteration = 393

Shrink model to first 394 iterations.
Learning rate set to 0.086395
0:	learn: 0.6394136	test: 0.6463440	best: 0.6463440 (0)	total: 1.75s	remaining: 1h 27m 32s
100:	learn: 0.6625279	test: 0.6892628	best: 0.6892681 (98)	total: 3m 16s	remaining: 1h 33m 51s
200:	learn: 0.6693477	test: 0.6922710	best: 0.6922904 (196)	total: 6m 16s	remaining: 1h 27m 21s
300:	learn: 0.6735453	test: 0.6961930	best: 0

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
# train start

scores = []
models = []


In [61]:
cat_features

['person_attribute_a_1',
 'person_attribute_b',
 'person_prefer_c',
 'person_prefer_d_1',
 'person_prefer_d_2',
 'person_prefer_d_3',
 'person_prefer_e',
 'person_prefer_h_1',
 'person_prefer_h_2',
 'person_prefer_h_3',
 'contents_attribute_i',
 'contents_attribute_a',
 'contents_attribute_j_1',
 'contents_attribute_c',
 'contents_attribute_l',
 'contents_attribute_d',
 'contents_attribute_m',
 'contents_attribute_e',
 'contents_attribute_h',
 'person_rn',
 'person_prefer_d_1_attribute_d_d',
 'person_prefer_d_1_attribute_d_s',
 'person_prefer_d_1_attribute_d_m',
 'person_prefer_d_1_attribute_d_l',
 'person_prefer_d_2_attribute_d_d',
 'person_prefer_d_2_attribute_d_s',
 'person_prefer_d_2_attribute_d_m',
 'person_prefer_d_2_attribute_d_l',
 'person_prefer_d_3_attribute_d_d',
 'person_prefer_d_3_attribute_d_s',
 'person_prefer_d_3_attribute_d_m',
 'person_prefer_d_3_attribute_d_l',
 'contents_attribute_d_attribute_d_d',
 'contents_attribute_d_attribute_d_s',
 'contents_attribute_d_attrib

{}

In [63]:
models = []
for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []

    model = CatBoostClassifier(iterations=iterations,random_state=SEED,eval_metric="F1",cat_features=cat_features,one_hot_max_size=4)
    model.fit(x_train.iloc[tri], y_train[tri],
            eval_set=[(x_train.iloc[vai], y_train[vai])],
            early_stopping_rounds=patience ,
            verbose = 100
        )

    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.086395
0:	learn: 0.5918078	test: 0.5960459	best: 0.5960459 (0)	total: 1.68s	remaining: 1h 24m 1s


KeyboardInterrupt: 